# FAIKR module 3 project

#Lorenzo Venieri
#Luca Zucchini

## Introduction

....

TODO:
-Aggiungere funzione per trovare correlazione tra features
-Considerare il fatto che il diabete causa durezza della pella, ma quest'ultima non è direttamente causa di diabete (=> != <=>)+
-Piccola spiegazione del ragionamento dietro le correlazioni NON trovate
-Fattore di rischio
-


In [ ]:
!pip install pgmpy

#prva

## Dataset

768 rows, 9 columns

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney
Diseases. The objective of the dataset is to diagnostically predict whether a patient has diabetes,
based on certain diagnostic measurements included in the dataset. Several constraints were placed
on the selection of these instances from a larger database. In particular, all patients here are females
at least 21 years old of Pima Indian heritage.2
From the data set in the (.csv) File We can find several variables, some of them are independent
(several medical predictor variables) and only one target dependent variable (Outcome).

In [6]:
import pandas as pd
import numpy as np

df_raw = pd.read_csv("data/diabetes.csv")
df_raw.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df_raw.corr()

In [13]:
df_raw.shape

(768, 9)

In [5]:
df_raw["Pregnancies"].unique()

array([ 6,  1,  8,  0,  5,  3, 10,  2,  4,  7,  9, 11, 13, 15, 17, 12, 14],
      dtype=int64)

In [6]:
df_raw.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

Vediamo che il dataset è sbilanciato

In [53]:
print("Numero di outcome seza diabete", (df_raw["Outcome"] == 0).sum())
print("Numero di outcome con diabete", (df_raw["Outcome"] == 1).sum())

Numero di outcome seza diabete 500
Numero di outcome con diabete 268


Vediamo che ci sono tati valori nulli nel dataset. Sarebbero da togliere dato che sono dei NaN. Allo stesso tempo possiamo usare questa cosa per iniziare un downsample dei dati con Outcome '0'

In [ ]:
for item in df_raw.columns:
    print("    - nella colonna ", item," sono presenti questi zeri")
    print((df_raw[item] == 0).sum())
    print("e questi sono tutti i valori disponibili:")
    temp = df_raw[item].unique()
    temp.sort()
    #print(df_raw[item].unique())
    print(temp)
    print("---------------------------")

In [ ]:
for item in df_raw.columns:
    counter = 0
    for i in range(len(df_raw[item])):
        if df_raw[item][i] == 0 and df_raw["Outcome"][i] == 0:
            counter += 1
    print("nella colonna", item,"ci sono ",counter,"righe da eliminare che possono semplicemente essere downsamplate")

In [ ]:
for item in df_raw.columns:
    print("valori di ",item)
    temp = df_raw[item].unique()
    temp.sort()
    #print(df_raw[item].unique())
    print(temp)
    print("----------------------")
    #
    #chiedere a Sten dei bin sensati per questi valori

Togliamo le righe che hanno un valore nullo aka NaN e un outcome '0'

In [54]:
df_clean = df_raw.copy(deep=True)

In [55]:
df_clean.shape

(768, 9)

In [56]:
columns_to_clean = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI','Age']
for item in columns_to_clean:
    drop_list = []
    for i in range(len(df_clean[item])):
        if df_clean[item][i] == 0 and df_clean["Outcome"][i] == 0:
            drop_list.append(i)
    print("SIUUUM")
    df_clean.drop(labels= drop_list, axis= 0, inplace= True)
    df_clean.reset_index(drop=True, inplace=True)


In [57]:
df_clean.shape

(530, 9)

Abbiamo tolto un po di roba, adiamo sia a vedere se e quanto è sbilanciato il dataset che a togliere le restati righe che hanno un NaN

In [64]:
print("Numero di outcome seza diabete", (df_clean["Outcome"] == 0).sum())
print("Numero di outcome con diabete", (df_clean["Outcome"] == 1).sum())

Numero di outcome seza diabete 262
Numero di outcome con diabete 268


Togliamo ora i restanti NaN senza però adare a toccare quelli dell'insulina ( presumiamo siano registrati nulli ) 

In [ ]:
for item in df_clean.columns:
    print("    - nella colonna ", item," sono presenti questi zeri")
    print((df_clean[item] == 0).sum())

In [ ]:
columns_to_clean = ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI','Age']

for item in columns_to_clean:
    drop_list = []
    for i in range(len(df_clean[item])):
        if df_clean[item][i] == 0 :
            drop_list.append(i)
    print("SIUUUM")
    df_clean.drop(labels= drop_list, axis= 0, inplace= True)
    df_clean.reset_index(drop=True, inplace=True)

Bene, andiamo a vedere cosa rimane del dataset originario

In [74]:
print("Numero di outcome seza diabete", (df_clean["Outcome"] == 0).sum())
print("Numero di outcome con diabete", (df_clean["Outcome"] == 1).sum())

Numero di outcome seza diabete 262
Numero di outcome con diabete 177


Come notiamo c'è ancora da fare un downsample per bilanciare il dataset

In [68]:
from sklearn.utils import resample

In [78]:
df_positive_outcome = df_clean[df_clean["Outcome"] == 1]
df_negative_outcome = df_clean[df_clean["Outcome"] == 0]

In [82]:
df_negative_outcome_downsampled = resample(df_negative_outcome,
             replace=True,
             n_samples=len(df_positive_outcome),
             random_state=42)
print(df_negative_outcome_downsampled.shape)

(177, 9)


In [85]:
df_balanced = pd.concat([df_negative_outcome_downsampled, df_positive_outcome])

In [86]:
print("Numero di outcome seza diabete", (df_balanced["Outcome"] == 0).sum())
print("Numero di outcome con diabete", (df_balanced["Outcome"] == 1).sum())


Numero di outcome seza diabete 177
Numero di outcome con diabete 177


In [ ]:
for item in df_balanced.columns:
    print("    - nella colonna ", item," sono presenti questi zeri")
    print((df_balanced[item] == 0).sum())

Bene, ora il dataset è bilanciato, non resta che binnarizzarlo

### Data Preprocessing


In [ ]:
#Printing the min and max values of age in order to better discretize the age parameter
print('Age range is: ', np.min(df_raw['Age']), ' - ', np.max(df_raw['Age']))

## Network


"In this chapter both the structure of the network, created from scratch by analyzing scientific papers related to the topic, and the parameter learning process are taken into account.

In particular, a variety of methods related to the analysis of Bayesian Networks are explored."

### Showing the network

A graphical preliminary overview of the network is shown using the visual library of pgmpy, the acronyms related to dataset attributes are the followings:

...


In [ ]:
!pip install daft

In [ ]:
import daft
import matplotlib.pyplot as plt
from daft import PGM

pgm = PGM(shape=[6, 7])

pgm.add_node(daft.Node('Gen', r"Gen", 2, 6))
pgm.add_node(daft.Node('Age', r"Age", 1, 6))
pgm.add_node(daft.Node('CPK', r"CPK", 3, 6))
pgm.add_node(daft.Node('Smo', r"Smo", 4, 6))
pgm.add_node(daft.Node('Sod', r"Sod", 5, 6))


pgm.add_node(daft.Node('Ana', r"Ana", 2, 5))

pgm.add_node(daft.Node('Dia', r"Dia", 1, 3))
pgm.add_node(daft.Node('Blo', r"Blo", 3, 3))

pgm.add_node(daft.Node('Fai', r"Fai", 3, 2))

pgm.add_node(daft.Node('Eje', r"Eje", 2, 1))
pgm.add_node(daft.Node('Pla', r"Pla", 4, 1))

pgm.add_edge('Gen', 'CPK')
pgm.add_edge('Gen', 'Dia')
pgm.add_edge('Age', 'Dia')
pgm.add_edge('Age', 'Ana')
pgm.add_edge('Gen', 'Ana')
pgm.add_edge('Dia', 'Blo')
pgm.add_edge('CPK', 'Blo')
pgm.add_edge('Ana', 'Blo')
pgm.add_edge('Smo', 'Blo')
pgm.add_edge('Sod', 'Blo')
pgm.add_edge('Blo', 'Fai')
pgm.add_edge('Dia', 'Fai')
pgm.add_edge('Fai', 'Eje')
pgm.add_edge('Fai', 'Pla')

pgm.render()
plt.show()


Examples of connections between nodes (e.g. direct cause, causal trail) refering to this specific Network are here shown:

#Direct cause

#Causal trail

#Common effect

## Creating the network

Connections between nodes have been implemented from scratch refering to a variety of scientific medical sources. Being the causal links in the medical field particularly challenging to model (i.e. often a huge variety of attributes are interlaced, causing directly or indirectly effects on each others) in this work only links that have been considered particularly relevant were defined.

Some of the connections are here explained and referenced:


In [ ]:
from pgmpy.models import BayesianModel

...

###Analyzing the network

Follows a series of experiments done on the network by applying a variety of methods provided by the pgmpy library in order to see in practice all the concepts addressed during the course.

In [ ]:
# Showing all the nodes of the model
model.nodes()

In [ ]:
# Showing all the edges of the model
model.edges()

In [ ]:
# Local independecies of a single node
model.local_independencies("high_blood_pressure")

In [ ]:
# Checking d-separation between variables with and without evidence
# Two sets of nodes X, Y are d-separated given Z if there is no active trail between any X ∈ X and Y ∈ Y given Z

print(model.is_dconnected("high_blood_pressure", "creatinine_phosphokinase"))
print(model.is_dconnected("serum_sodium", "diabetes",observed=["anaemia"] ))

In [ ]:
# Function is_irrelevant made from scratch by the definition of irrelevance given during the course 

def is_irrelevant (node1,node2,evidence):
  if ((node2 in (model.get_ancestral_graph(node1)and model.get_ancestral_graph(evidence))) and model.is_dconnected(node1, node2,observed=evidence )):
    print("The node {0} is not irrelevant with resepect to {1}, given the evidence {2}.".format(node1, node2, evidence))
  else:
    print("The node {0} is irrelevant with resepect to {1}, given the evidence {2}".format(node1, node2, evidence))


is_irrelevant("ejection_fraction", "diabetes", "heart_failure")

In [ ]:
# Checking active trails from high blood pressure given the evidence diabetes
model.active_trail_nodes('high_blood_pressure', observed='diabetes')

In [ ]:
# Checking the markov blanket of the node heart failure
model.get_markov_blanket("heart_failure")

### Parameter learning

Learning Bayesian networks from data, knowing the structure of the network, boils down to parameter estimation. In pgmpy a variety of estimators is available, going from IVestimator to SEMestimator, but the main choice for this problem fell on two of them: Maximum Likelihood and Bayesian Estimator.

MLE, which depends solely on the outcomes of observed data, could be a reasonable simple starting point, however, it is notorious for becoming easily biased when the data is minimal. Moreover, in situations where observed data is sparse, Bayesian estimation’s incorporation of prior knowledge can help in attaining a more accurate model. 

On the other hand, unreliable priors can lead to a slippery slope of highly biased models that require large amounts of seen data to remedy; so priors need to be well defined and contain relevant insight to the problem in order to avoid that. 

Taken into consideration pros and cons of both approaches, given the minimal dataset used for this work the Bayesian Estimator is choosen.

Pgmpy also offer a variety of possible priors, among them a possible choice is Bayesian Dirichlet equivalent uniform prior (BDeu), choosen with its default settings (i.e. equivalent_sample_size equal to 5).

In the following cell the parameter estimation process takes place and the learned CPTs are shown.

In [ ]:
from pgmpy.estimators import BayesianEstimator

model.cpds = []
model.fit(data=df, estimator=BayesianEstimator, prior_type="BDeu")

for cpd in model.get_cpds():
    print(f'CPT of {cpd.variable}:')
    print(cpd, '\n')

In [ ]:
# Checking the validity of the model 
# This method checks if the sum of the probabilities for each state is equal to 1 (tol=0.01) and if the CPDs associated with nodes are consistent with their parents.
model.check_model()